In [1]:
from torchvision import transforms, datasets

datasets.FashionMNIST("data", train=True, download=True)
datasets.FashionMNIST("data", train=False, download=True)

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchvision
import torchvision.models.resnet
import os
import time
import argparse
import random
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
from pytorch_lightning import Trainer
# %pip install wandb
import wandb

# %pip install lightning
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint

In [3]:

class ConvNet(nn.Module):
    def __init__(self, dropout=0.25):
        super(ConvNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1),
            nn.ReLU(),
            nn.Dropout2d(dropout) if dropout else nn.Identity(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.ReLU(),
            nn.Dropout2d(dropout) if dropout else nn.Identity(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(dropout) if dropout else nn.Identity(),
            nn.Linear(128, 10)
        )

        self.apply(self.init_weights)

    def init_weights(self, m):
        if isinstance(m, nn.Conv2d):
            init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            m.bias.data.fill_(0.00)
        elif isinstance(m, nn.Linear):
            init.kaiming_uniform_(m.weight, nonlinearity='relu')
            m.bias.data.fill_(0.00)
        # if fc2 is the last layer, we can use the following initialization
        if m == self.layers[-1]:
            init.xavier_uniform_(m.weight, gain=init.calculate_gain('linear'))

    def forward(self, x):
        x = self.layers(x)
        return x



In [ ]:
class MNISTData(L.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.train_set = None
        self.val_set = None
        self.test_set = None
        self.batch_size = batch_size

    def prepare_data(self) -> None:
        train_set = datasets.FashionMNIST("data", train=True, download=True,
                                          transform=transforms.Compose([transforms.ToTensor()]))
        self.test_set = datasets.FashionMNIST("data", train=False, download=True,
                                              transform=transforms.Compose([transforms.ToTensor()]))
        self.train_set, self.val_set = torch.utils.data.random_split(train_set, [50000, 10000])

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=15, persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, num_workers=15, persistent_workers=True)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, num_workers=15, persistent_workers=True)


class Classifier(L.LightningModule):
    def __init__(self, model=None, lr=0.001, dropout=0.25, weight_decay=1e-5):
        super().__init__()
        self.lr = lr
        self.weight_decay = weight_decay
        self.model = model if model is not None else ConvNet(dropout)
        self.save_hyperparameters()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
        self.log_dict({"val/loss": loss, "val/accuracy": accuracy})
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
        self.log_dict({"test/loss": loss, "test/accuracy": accuracy})
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)


# Baseline

In [5]:
torch.manual_seed(42)

data_module = MNISTData()
convnet_classifier = Classifier(lr=0.001, dropout=0.25)

wandb_logger = WandbLogger(project="fashion_mnist_project")


trainer = L.Trainer(max_epochs=10, accelerator='gpu', devices=1, logger=wandb_logger)


trainer.fit(convnet_classifier, data_module)
metrics = trainer.logged_metrics["val/accuracy"]
test_results = trainer.test(convnet_classifier, datamodule=data_module)
print("Test Results:", test_results)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ziele14 (ziele14-poznan-univeristy-of-technology). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | ConvNet | 421 K  | train
------------------------------------------
421 K     Trainable params
0         Non-trainable params
421 K     Total params
1.687     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|██████████| 782/782 [00:05<00:00, 130.72it/s, v_num=z24l]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 782/782 [00:06<00:00, 130.22it/s, v_num=z24l]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\mateu\anaconda3\envs\pytorch\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 157/157 [00:01<00:00, 105.57it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test/accuracy          0.911899983882904
        test/loss           0.24953396618366241
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test Results: [{'test/loss': 0.24953396618366241, 'test/accuracy': 0.911899983882904}]


In [6]:
torch.save(convnet_classifier.state_dict(), "convnet_classifier.pth")



# Quantized dynamic

In [ ]:
quantized_model_dynamic = Classifier(lr=0.001, dropout=0.25)
quantized_model_dynamic.load_state_dict(torch.load("convnet_classifier.pth"))
quantized_model_dynamic.model = torch.quantization.quantize_dynamic(
    quantized_model_dynamic.model,  
    {torch.nn.Linear}, 
    dtype=torch.qint8  
)

# Pruning weights

In [5]:
import torch.nn.utils.prune as prune

In [ ]:
pruned_model = Classifier(lr=0.001, dropout=0.25)
pruned_model.load_state_dict(torch.load("convnet_classifier.pth"))
parameters_to_prune = (
    (pruned_model.model.layers[0], 'weight'),
    (pruned_model.model.layers[4], 'weight'),
    (pruned_model.model.layers[9], 'weight'),
)
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

for module, param in parameters_to_prune:
    prune.remove(module, param)

# Utils

In [7]:
def measure_inference_time(model, dataloader, num_warmup=5):
    model.eval()
    
    # warm ups at the start so that the measurements are not skewed
    with torch.no_grad():
        for _ in range(num_warmup):
            for x, y in dataloader:
                model(x)
    
    start_time = time.perf_counter()
    with torch.no_grad():
        for x, y in dataloader:
            model(x)
    end_time = time.perf_counter()
    
    return (end_time - start_time) / len(dataloader)


In [8]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size


In [9]:
def model_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [13]:
print_size_of_model(convnet_classifier, "original")


model:  original  	 Size (KB): 1689.628


1689628

In [14]:
model_parameters(convnet_classifier)

421642

In [15]:
print_size_of_model(quantized_model_dynamic, "quantized")

model:  quantized  	 Size (KB): 482.946


482946

In [16]:
model_parameters(quantized_model_dynamic)

18816

In [ ]:
print_size_of_model(pruned_model, "pruned") 


model:  pruned  	 Size (KB): 1689.628


1689628

In [11]:
model_parameters(pruned_model) #because the weights are zeroed not removed

421642

In [19]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [14]:
def evaluate(model, data_loader):
    model.eval()  
    all_preds = []
    all_labels = []


    with torch.no_grad():
        for x, y in data_loader:
            outputs = model(x)

            preds = torch.argmax(outputs, dim=1)


            all_preds.append(preds.cpu().numpy())  
            all_labels.append(y.cpu().numpy())    

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)


    f1 = f1_score(all_labels, all_preds, average='weighted')  
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    return {"f1":f1, "accuracy":accuracy, "precision":precision, "recall":recall}

In [21]:
f1,ac,pr,rc = evaluate(convnet_classifier, data_module.test_dataloader())

# Comparison

In [22]:
inference_times ={}
inference_times["baseline"] = measure_inference_time(convnet_classifier, data_module.test_dataloader())
inference_times["quantized_dynamic"] = measure_inference_time(quantized_model_dynamic, data_module.test_dataloader())
inference_times["pruned"] = measure_inference_time(pruned_model, data_module.test_dataloader())

In [ ]:
evaluation_metrics = {}
evaluation_metrics["baseline"] = evaluate(convnet_classifier, data_module.test_dataloader())
evaluation_metrics["quantized_dynamic"] = evaluate(quantized_model_dynamic, data_module.test_dataloader())
evaluation_metrics["pruned"] = evaluate(pruned_model, data_module.test_dataloader())

In [24]:
evaluation_metrics

{'baseline': {'f1': np.float64(0.9110210839126303),
  'accuracy': 0.9119,
  'precision': np.float64(0.9131507923675721),
  'recall': np.float64(0.9119)},
 'quantized_dynamic': {'f1': np.float64(0.9119113933269416),
  'accuracy': 0.9128,
  'precision': np.float64(0.9139543236372445),
  'recall': np.float64(0.9128)},
 'pruned': {'f1': np.float64(0.9111225528617409),
  'accuracy': 0.912,
  'precision': np.float64(0.9132195084547012),
  'recall': np.float64(0.912)}}

In [25]:
inference_times

{'baseline': 0.012010881528599414,
 'quantized_dynamic': 0.01177263503173473,
 'pruned': 0.01190352292989446}

In [26]:
import pandas as pd
metrics_df = pd.DataFrame(evaluation_metrics).T.reset_index().rename(columns={'index': 'Model'})
times_df = pd.DataFrame(inference_times, index=[0]).T.reset_index().rename(columns={'index': 'Model', 0: 'Inference Time'})

result_df = pd.merge(metrics_df, times_df, on='Model')

result_df

,Model,f1,accuracy,precision,recall,Inference Time
0,baseline,0.911021,0.9119,0.913151,0.9119,0.012011
1,quantized_dynamic,0.911911,0.9128,0.913954,0.9128,0.011773
2,pruned,0.911123,0.9120,0.913220,0.9120,0.011904


# Knowledge distillation

In [27]:
class LightConvNet(nn.Module):
    def __init__(self, dropout=0.25):
        super(LightConvNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 8, 3, 1, 1),
            nn.ReLU(),
            nn.Dropout2d(dropout) if dropout else nn.Identity(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 3, 1, 1),
            nn.ReLU(),
            nn.Dropout2d(dropout) if dropout else nn.Identity(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(16 * 7 * 7, 32),
            nn.ReLU(),
            nn.Dropout(dropout) if dropout else nn.Identity(),
            nn.Linear(32, 10)
        )

        self.apply(self.init_weights)
        
    def init_weights(self, m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        return self.layers(x)

In [ ]:
torch.manual_seed(42)
student_model_one = Classifier(model=LightConvNet(), lr=0.001, dropout=0.25)

torch.manual_seed(42)
second_student_model = Classifier(model=LightConvNet(), lr=0.001, dropout=0.25)

In [29]:
model_parameters(student_model_one)

26698

In [30]:
model_parameters(second_student_model)

26698

In [31]:
# wandb_logger = WandbLogger(project="fashion_mnist_project")
trainer = L.Trainer(max_epochs=10, accelerator='gpu', devices=1, logger=wandb_logger)
trainer.fit(student_model_one, data_module)
metrics = trainer.logged_metrics["val/accuracy"]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\mateu\anaconda3\envs\pytorch\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory .\fashion_mnist_project\yugcz24l\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | LightConvNet | 26.7 K | train
-----------------------------------------------
26.7 K    Trainable params
0         Non-trainable params
26.7 K    Total params
0.107     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|██████████| 782/782 [00:06<00:00, 117.68it/s, v_num=z24l]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 782/782 [00:06<00:00, 117.46it/s, v_num=z24l]


In [32]:
test_results_light = trainer.test(student_model_one, datamodule=data_module)
print("Test Results:", test_results_light)
print("Test Results trainer:", test_results)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\mateu\anaconda3\envs\pytorch\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 157/157 [00:01<00:00, 97.15it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test/accuracy         0.8840000033378601
        test/loss           0.3209458589553833
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test Results: [{'test/loss': 0.3209458589553833, 'test/accuracy': 0.8840000033378601}]
Test Results trainer: [{'test/loss': 0.24953396618366241, 'test/accuracy': 0.911899983882904}]


In [33]:
torch.save(student_model_one.state_dict(), "student.pth")

In [42]:
class Classifier_distillation(L.LightningModule):
    def __init__(self, model=None, teacher=None, lr=0.001, dropout=0.25, weight_decay=1e-5, T=2, soft_target_loss_weight=0.25, ce_loss_weight=0.75):
        super().__init__()
        self.lr = lr
        self.weight_decay = weight_decay
        self.model = model if model is not None else ConvNet(dropout)
        self.teacher = teacher
        self.T = T
        self.soft_target_loss_weight = soft_target_loss_weight
        self.ce_loss_weight = ce_loss_weight
        self.ce_loss = nn.CrossEntropyLoss()
        self.save_hyperparameters()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        student_logits = self(x)
        label_loss = self.ce_loss(student_logits, y)

        if self.teacher:
            with torch.no_grad():
                teacher_logits = self.teacher(x)
            soft_targets = nn.functional.softmax(teacher_logits / self.T, dim=-1)
            soft_prob = nn.functional.log_softmax(student_logits / self.T, dim=-1)
            soft_targets_loss = torch.sum(soft_targets * (soft_targets.log() - soft_prob)) / soft_prob.size()[0] * (self.T**2)
            loss = self.soft_target_loss_weight * soft_targets_loss + self.ce_loss_weight * label_loss
        else:
            loss = label_loss

        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/accuracy", accuracy, prog_bar=True)
        return {"val_loss": loss, "val_accuracy": accuracy}

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
        self.log_dict({"test/loss": loss, "test/accuracy": accuracy})
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

In [43]:
# student_model_one.load_state_dict(torch.load("student.pth"))

In [ ]:
teacher_model = convnet_classifier
student_model = second_student_model
classifier = Classifier_distillation(model=student_model, teacher=teacher_model)

# wandb_logger = WandbLogger(project="fashion_mnist_project")
trainer = L.Trainer(max_epochs=10, accelerator='gpu', devices=1, logger=wandb_logger)

trainer.fit(classifier, data_module)

c:\Users\mateu\anaconda3\envs\pytorch\lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
c:\Users\mateu\anaconda3\envs\pytorch\lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'teacher' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['teacher'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\mateu\anaconda3\envs\pytorch\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory .\fashion_mnist_project\yugcz24l\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------

Epoch 9: 100%|██████████| 782/782 [00:08<00:00, 97.10it/s, v_num=z24l, val/loss=0.229, val/accuracy=0.916] 

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 782/782 [00:08<00:00, 96.87it/s, v_num=z24l, val/loss=0.229, val/accuracy=0.916]


In [ ]:
test_results_distilled = trainer.test(classifier, datamodule=data_module)
print("Test Results:", test_results_distilled)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\mateu\anaconda3\envs\pytorch\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 157/157 [00:01<00:00, 99.76it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test/accuracy         0.8996999859809875
        test/loss           0.2757464349269867
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test Results: [{'test/loss': 0.2757464349269867, 'test/accuracy': 0.8996999859809875}]


In [49]:
torch.save(classifier.state_dict(), "student_distilled.pth")

In [46]:
convnet_metrics = test_results
convnet_inference_time = measure_inference_time(convnet_classifier, data_module.test_dataloader())

student_1_metrics = test_results_light
student_1_inference_time = measure_inference_time(student_model_one, data_module.test_dataloader())

student_2_metrics = trainer.test(classifier, datamodule=data_module)
student_2_inference_time = measure_inference_time(classifier, data_module.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 157/157 [00:01<00:00, 103.15it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test/accuracy         0.8996999859809875
        test/loss           0.2757464349269867
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [47]:
evaluation_metrics = {}
evaluation_metrics["baseline"] = evaluate(convnet_classifier, data_module.test_dataloader())
evaluation_metrics["student_one"] = evaluate(student_model_one, data_module.test_dataloader())
evaluation_metrics["student_two"] = evaluate(classifier, data_module.test_dataloader())

In [48]:
results = {
    'Model': ['ConvNet baseline', 'Student Model 1', 'Student Model 2'],
    'Accuracy': [convnet_metrics[0]['test/accuracy'], student_1_metrics[0]['test/accuracy'], student_2_metrics[0]['test/accuracy']],
    'Loss': [convnet_metrics[0]['test/loss'], student_1_metrics[0]['test/loss'], student_2_metrics[0]['test/loss']],
    'Precision': [
        evaluation_metrics["baseline"]["precision"],
        evaluation_metrics["student_one"]["precision"],
        evaluation_metrics["student_two"]["precision"]
    ],
    'Recall': [
        evaluation_metrics["baseline"]["recall"],
        evaluation_metrics["student_one"]["recall"],
        evaluation_metrics["student_two"]["recall"]
    ],
    'F1 Score': [
        evaluation_metrics["baseline"]["f1"],
        evaluation_metrics["student_one"]["f1"],
        evaluation_metrics["student_two"]["f1"]
    ],
    'Inference Time (s)': [convnet_inference_time, student_1_inference_time, student_2_inference_time]
}

results_df_two = pd.DataFrame(results)

results_df_two

,Model,Accuracy,Loss,Precision,Recall,F1 Score,Inference Time (s)
0,ConvNet baseline,0.9119,0.249534,0.913151,0.9119,0.911021,0.012287
1,Student Model 1,0.8840,0.320946,0.883589,0.8840,0.883581,0.007893
2,Student Model 2,0.8997,0.275746,0.899897,0.8997,0.899260,0.008605
